### Verification of Fitting Regression Models
Bandwidth of Frequencies (Multiple Input - Single Output)  
This notebook is optimized for SOH estimation

**In order to see the results you should start the MLflow ui seperately**:
1. Open a promt/terminal and navigate to the path of this project
2. Activate the virtual environment:  
    (Windows: ```.venv\eis_data_analytics\Scripts\activate```,  
    Linux/Mac: ```.venv/eis_data_analytics/bin/activate```)
3. Now start MLflow with ```mlflow server --port 1234``` consider to add e.g.: ```--workers=16 --gunicorn-opts='--timeout 600'```
4. Open [http://127.0.0.1:1234](http://127.0.0.1:1234) in your browser

In [ ]:
%matplotlib widget 

from modules import data_preparation as dp
from modules import dataset_manipulation as dm
from modules import eisplot as eisplot
from modules.eisplot import plt
from modules.eisplot import mpl

import numpy as np
import pandas as pd

from functools import partial
import hyperopt
import mlflow
import shapely
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR


cm = 1 / 2.54  # centimeters in inches

## if you have installed latex and want to use it for plots, uncomment the following 3 lines
# eisplot.mpl.rcParams.update({"text.usetex": True,'savefig.format':'pdf'})
# eisplot.mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# eisplot.mpl.rc('text.latex', preamble=r'\usepackage{underscore}')

## safe figures e.g. with:
# plot_name = "custom_3D_plot"
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

### Load Data

In [ ]:
name_of_this_run = "reference_data"

destination_filepath = r"./data/eis_datasets/" + name_of_this_run + ".parquet"
df = pd.read_parquet(destination_filepath)
destination_filepath = r"./data/key_lookup/key_lookup_" + name_of_this_run + ".parquet"
key_lookup_df = pd.read_parquet(destination_filepath)

In [ ]:
name_of_the_feature = "_abs"
# name_of_the_feature = "_abs_temp_soc"
# name_of_the_feature = "_abs_phase_temp_soc"

feature_selection = dm.json_2_list(name_of_this_run + name_of_the_feature + ".json")
print(feature_selection)

### Define the Output Parameter of the Model

In [ ]:
output_parameter = "SOH"

In [ ]:
experiment_name = (
    name_of_this_run + "_" + "SVR_MISO_Random_" + output_parameter + name_of_the_feature
)
mlflow_exp = mlflow.set_experiment(experiment_name=experiment_name)

### Define if Arrenhius prescaling should be used

In [ ]:
def arrhenius_correction(value):
    return np.log(1 / value)


def arrhenius_correction_inverse(value):
    return 1 / np.exp(value)

In [ ]:
[arrhenius_correction_inverse(arrhenius_correction(i)) for i in [0.01, 0.1, 1, 10]]

Here we directly modify the database

In [ ]:
df[key_lookup_df["EIS_Z_abs"].to_list()] = arrhenius_correction(
    df[key_lookup_df["EIS_Z_abs"].to_list()].values
)

### Convert to Training Data and define Calculation of Errors

The data is split 20 / 80 % into validation and train data. As 150 is out of the value range the test set is just empty.

In [ ]:
# convert to training arrays
data_set = dm.get_set(
    df,
    output_parameter,
    feature_keys=feature_selection,
    validation_split=0.2,
    output_intervals_for_test=[[150, 151]],
)
x_train, y_train = data_set["train"]
x_validation, y_validation = data_set["validation"]
x_test, y_test = data_set["test"]

### The SVR

In [ ]:
transformer = MinMaxScaler()
transformer = transformer.fit(x_train)

In [ ]:
def evaluate_mse_svr(model, x, y):
    y_pred = model.predict(x)
    y_orig = y
    mse = ((y_pred - y_orig) ** 2).mean()
    return mse

#### For Hyperparameter optimization the fmin of hyperopt is used

Define an objective to be minimized

In [ ]:
def svm_objective(params, experiment_id):
    """
    Optimize the SVM model using SVR.

    Parameters:
        params (dict): The parameters for the SVM model and the validation.
        experiment_id: The id of the mlflow experiment.

    Returns:
        dict: The loss and status of the optimization.
    """

    default_params = {
        "log_model": False,
        "plot_diag": False,
        "log_plot_type": "png",
        "kernel": "rbf",
        "epsilon": 0.1,
        "C": 1,
        "tol": 0.001,
        "coef0": 0.0,
        "gamma": "auto",
        "degree": 3,
    }

    default_params.update(params)
    merged_params = default_params

    apply_test = False

    with mlflow.start_run(experiment_id=experiment_id, run_name="SVR"):
        if merged_params["log_model"]:
            mlflow.sklearn.autolog()

        mlflow.log_param("kernel", merged_params["kernel"])
        mlflow.log_param("C", merged_params["C"])
        mlflow.log_param("tol", merged_params["tol"])
        mlflow.log_param("gamma", merged_params["gamma"])
        mlflow.log_param("epsilon", merged_params["epsilon"])
        mlflow.log_param("degree", merged_params["degree"])
        mlflow.log_param("coef0", merged_params["coef0"])

        pipeline = Pipeline(
            steps=[
                ("scaler", transformer),
                (
                    "svm",
                    SVR(
                        kernel=merged_params["kernel"],
                        C=merged_params["C"],
                        tol=merged_params["tol"],
                        gamma=merged_params["gamma"],
                        epsilon=merged_params["epsilon"],
                        degree=merged_params["degree"],
                        coef0=merged_params["coef0"],
                        cache_size=4000,
                    ),
                ),
            ]
        )

        pipeline.fit(x_train, y_train)
        pipeline.score(x_validation, y_validation)

        train_mse_temp = evaluate_mse_svr(pipeline, x_train, y_train)
        validation_mse_temp = evaluate_mse_svr(pipeline, x_validation, y_validation)
        if apply_test:
            test_mse_temp = evaluate_mse_svr(pipeline, x_test, y_test)

        mlflow.log_metric("train_mse_temp", train_mse_temp)
        mlflow.log_metric("validation_mse_temp", validation_mse_temp)
        if apply_test:
            mlflow.log_metric("test_mse_temp", test_mse_temp)

        if merged_params["plot_diag"]:
            fig, ax = plt.subplots(1, 1, figsize=(7 * cm, 7 * cm))
            plt.cla()

            # prediction on train set
            y_pred = pipeline.predict(x_train)
            cell_list = list(set(data_set["df_train"].index.get_level_values(0)))
            fig, ax = eisplot.cell_scatter(
                data_set,
                y_pred,
                cell_names=cell_list,
                title=False,
                legend=False,
                fig=fig,
                ax=ax,
            )

            # prediction on validation set
            y_pred = pipeline.predict(x_validation)
            cell_list = list(set(data_set["df_validation"].index.get_level_values(0)))
            fig, ax = eisplot.cell_scatter(
                data_set,
                y_pred,
                is_validation=True,
                cell_names=cell_list,
                title=False,
                legend=False,
                fig=fig,
                ax=ax,
                add_trendline=False,
            )

            if apply_test:
                # prediction on test set
                y_pred = pipeline.predict(x_test)
                cell_list = list(set(data_set["df_test"].index.get_level_values(0)))
                fig, ax = eisplot.cell_scatter(
                    data_set,
                    y_pred,
                    is_test=True,
                    cell_names=cell_list,
                    title=False,
                    legend=False,
                    fig=fig,
                    ax=ax,
                    add_trendline=False,
                )

            if (name_of_this_run == "example_data") & (
                output_parameter == "Temperature"
            ):
                ax.set_xlim([-30, 60])
                ax.set_ylim([-30, 60])

                ax.text(
                    -4,
                    -19,
                    "Train MSE: " + "%.2f" % train_mse_temp + " K",
                    horizontalalignment="left",
                    verticalalignment="center",
                    fontsize=8,
                )
                ax.text(
                    -4,
                    -23,
                    "Validation MSE: " + "%.2f" % validation_mse_temp + " K",
                    horizontalalignment="left",
                    verticalalignment="center",
                    fontsize=8,
                )
                if apply_test:
                    ax.text(
                        -4,
                        -27,
                        "Test MSE: " + "%.2f" % test_mse_temp + " K",
                        horizontalalignment="left",
                        verticalalignment="center",
                        fontsize=8,
                    )

            legend_elements = [
                mpl.lines.Line2D(
                    [0],
                    [0],
                    color=eisplot.rwth_colors.colors[("green", 100)],
                    label="ideal",
                ),
                mpl.lines.Line2D(
                    [0],
                    [0],
                    marker=".",
                    color=eisplot.rwth_colors.colors[("blue", 100)],
                    linestyle="",
                    label="train",
                    alpha=0.5,
                ),
                mpl.lines.Line2D(
                    [0],
                    [0],
                    marker="2",
                    color=eisplot.rwth_colors.colors[("orange", 100)],
                    linestyle="",
                    label="validation",
                    alpha=0.5,
                ),
                mpl.lines.Line2D(
                    [0],
                    [0],
                    marker="1",
                    color=eisplot.rwth_colors.colors[("lavender", 100)],
                    linestyle="",
                    label="test",
                    alpha=0.5,
                ),
            ]
            if apply_test:
                ax.legend(
                    handles=legend_elements,
                    loc="best",
                    scatterpoints=1,
                    prop={"size": 8},
                )
            else:
                ax.legend(
                    handles=legend_elements[:-1],
                    loc="best",
                    scatterpoints=1,
                    prop={"size": 8},
                )
            fig.subplots_adjust(bottom=0.14, left=0.19)
            mlflow.log_figure(
                fig, "prediction_vs_actual." + merged_params["log_plot_type"]
            )
            plt.close()

        mlflow.log_metric("support_vectors", pipeline.named_steps["svm"].n_support_)
        mlflow.log_metric(
            "support_vectors_percent",
            pipeline.named_steps["svm"].n_support_ / x_train.shape[0],
        )
        if apply_test:
            mlflow.log_metric(
                "std_mse", np.std([train_mse_temp, validation_mse_temp, test_mse_temp])
            )
            mlflow.log_metric(
                "max_mse", np.max([train_mse_temp, validation_mse_temp, test_mse_temp])
            )
            mlflow.log_metric(
                "std_times_max_mse",
                np.std([train_mse_temp, validation_mse_temp, test_mse_temp])
                * np.max([train_mse_temp, validation_mse_temp, test_mse_temp]),
            )
        else:
            mlflow.log_metric("std_mse", np.std([train_mse_temp, validation_mse_temp]))
            mlflow.log_metric("max_mse", np.max([train_mse_temp, validation_mse_temp]))
            mlflow.log_metric(
                "std_times_max_mse",
                np.std([train_mse_temp, validation_mse_temp])
                * np.max([train_mse_temp, validation_mse_temp]),
            )

        # fmin() minimizes the objective
        if apply_test:
            weighted_fit_result = np.max(
                [train_mse_temp, validation_mse_temp, test_mse_temp]
            )
        else:
            weighted_fit_result = np.max([train_mse_temp, validation_mse_temp])

    return {"loss": weighted_fit_result, "status": hyperopt.STATUS_OK}

In [ ]:
# search_space = hyperopt.hp.choice('SVM', [
#     {
#         'log_model': hyperopt.hp.choice('log_model', [True]),
#         'plot_diag': hyperopt.hp.choice('plot_diag', [True]),
#         'log_plot_type': hyperopt.hp.choice('log_plot_type', ['svg']),
#         'gamma': hyperopt.hp.choice('gamma', ['scale']),
#         'tol': hyperopt.hp.choice('tol', [0.001]),
#         'C': hyperopt.hp.choice('C', [0.01, 0.1, 1, 10, 100, 1000]),
#         'epsilon': hyperopt.hp.choice('epsilon', [0.1]),
#         'degree': hyperopt.hp.choice('degree', [3, 4, 5]),
#         'coef0': hyperopt.hp.choice('coef0', [0.0]),
#         'kernel': hyperopt.hp.choice('kernel', ['rbf','linear','poly']),
#     }
# ])

search_space = hyperopt.hp.choice(
    "SVM",
    [
        {
            "log_model": hyperopt.hp.choice("log_model", [True]),
            "plot_diag": hyperopt.hp.choice("plot_diag", [True]),
            "log_plot_type": hyperopt.hp.choice("log_plot_type", ["svg"]),
            "gamma": hyperopt.hp.loguniform("gamma", np.log(0.001), np.log(100)),
            "tol": hyperopt.hp.loguniform("tol", np.log(0.001), np.log(10)),
            "C": hyperopt.hp.loguniform("C", np.log(0.01), np.log(10000000000)),
            "epsilon": hyperopt.hp.loguniform("epsilon", np.log(0.01), np.log(10)),
            "kernel": hyperopt.hp.choice("kernel", ["rbf"]),
        }
    ],
)

Choose an optimization type

In [ ]:
# algo=hyperopt.tpe.suggest
algo = hyperopt.rand.suggest

#### Fit the model, you can track it in mlflow: [http://127.0.0.1:1234](http://127.0.0.1:1234)

In [ ]:
timout_in_minutes = 24 * 60
max_evals = 10

## if java is installed (only recommended under linux/wsl)
# import pyspark
# spark_trails = hyperopt.SparkTrials(parallelism=4)
# best_result = hyperopt.fmin(fn=partial(svm_objective,experiment_id=mlflow_exp.experiment_id),
#                             space=search_space,
#                             algo=algo,
#                             max_evals=max_evals,
#                             timeout=timout_in_minutes*60,
#                             trials=spark_trails)
# if java is not available
best_result = hyperopt.fmin(
    fn=partial(svm_objective, experiment_id=mlflow_exp.experiment_id),
    space=search_space,
    algo=algo,
    max_evals=max_evals,
    timeout=timout_in_minutes * 60,
)

In [ ]:
print(hyperopt.space_eval(search_space, best_result))

### Validate the model

In [ ]:
name_of_this_run_eval = name_of_this_run

destination_filepath = r"./data/eis_datasets/" + name_of_this_run_eval + ".parquet"
df_eval = pd.read_parquet(destination_filepath)

Direct scaling on the dataset

In [ ]:
df_eval[key_lookup_df["EIS_Z_abs"].to_list()] = arrhenius_correction(
    df_eval[key_lookup_df["EIS_Z_abs"].to_list()].values
)

In [ ]:
feature_selection_eval = feature_selection

In [ ]:
output_parameter_eval = output_parameter

In [ ]:
apply_test = False

In [ ]:
# convert to arrays
data_set_eval = dm.get_set(
    df_eval,
    output_parameter_eval,
    feature_keys=feature_selection_eval,
    validation_split=0.2,
    output_intervals_for_test=[[150, 151]],
)
x_train_eval, y_train_eval = data_set_eval["train"]
x_validation_eval, y_validation_eval = data_set_eval["validation"]
x_test_eval, y_test_eval = data_set_eval["test"]

Open [http://127.0.0.1:1234](http://127.0.0.1:1234) to select a fitted model. If you click on it, you can extract the run ID. It could look like this: "ad26474e8c324f84906c9fc501928cae"

In [ ]:
# you can choose a specific model
# logged_model = 'ad26474e8c324f84906c9fc501928cae'
# or just load the best model
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id],
    order_by=["metrics.max_mse"],
    max_results=1,
)
logged_model = runs[0].info.run_id

# Load model as a PyFuncModel.
run_eval = mlflow.get_run(logged_model)
loaded_model = mlflow.pyfunc.load_model(run_eval.info.artifact_uri + "/model/")

train_mse_temp = evaluate_mse_svr(loaded_model, x_train_eval, y_train_eval)
print("Train MSE: " + str(train_mse_temp))
validation_mse_temp = evaluate_mse_svr(
    loaded_model, x_validation_eval, y_validation_eval
)
print("Validation MSE: " + str(validation_mse_temp))
if apply_test:
    test_mse_temp = evaluate_mse_svr(loaded_model, x_test_eval, y_test_eval)
    print("Test MSE: " + str(test_mse_temp))

print(
    "C: "
    + run_eval.data.params["svm__C"]
    + ", epsilon: "
    + run_eval.data.params["svm__epsilon"]
    + ", tol: "
    + run_eval.data.params["svm__tol"]
    + ", Gamma: "
    + run_eval.data.params["svm__gamma"]
)

Plot the Result

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10 * cm, 10 * cm))

cell_list = list(set(data_set_eval["df_train"].index.get_level_values(0)))
y_pred_train_eval = loaded_model.predict(x_train_eval)
fig, ax = eisplot.cell_scatter(
    data_set_eval, y_pred_train_eval, cell_names=cell_list, fig=fig, ax=ax
)

cell_list = list(set(data_set_eval["df_validation"].index.get_level_values(0)))
y_pred_validation_eval = loaded_model.predict(x_validation_eval)
fig, ax = eisplot.cell_scatter(
    data_set_eval,
    y_pred_validation_eval,
    cell_names=cell_list,
    fig=fig,
    ax=ax,
    is_validation=True,
)

if apply_test:
    cell_list = list(set(data_set_eval["df_test"].index.get_level_values(0)))
    y_pred_test_eval = loaded_model.predict(x_test_eval)
    fig, ax = eisplot.cell_scatter(
        data_set_eval,
        y_pred_test_eval,
        cell_names=cell_list,
        fig=fig,
        ax=ax,
        is_test=True,
    )


legend_elements = [
    mpl.lines.Line2D(
        [0], [0], color=eisplot.rwth_colors.colors[("green", 100)], label="ideal"
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker=".",
        linestyle="",
        color=eisplot.rwth_colors.colors[("blue", 100)],
        label="train",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="2",
        linestyle="",
        color=eisplot.rwth_colors.colors[("orange", 100)],
        label="validation",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="1",
        linestyle="",
        color=eisplot.rwth_colors.colors[("lavender", 100)],
        label="test",
        alpha=0.5,
    ),
]
if apply_test:
    ax.legend(handles=legend_elements, loc="best", scatterpoints=1, prop={"size": 8})
else:
    ax.legend(
        handles=legend_elements[:-1], loc="best", scatterpoints=1, prop={"size": 8}
    )
fig.subplots_adjust(bottom=0.14, left=0.19)

If this result is not the best, further analysis of the results is required

In [ ]:
experiment_list = pd.DataFrame()

In [ ]:
experiment_id = mlflow_exp.experiment_id
experiment_id

In [ ]:
for exp in mlflow.search_experiments():
    if exp.experiment_id == experiment_id:
        experiment_tmp = mlflow.search_runs(experiment_ids=experiment_id)
        experiment_list = pd.concat([experiment_list, experiment_tmp])

In [ ]:
experiment_list = experiment_list.reset_index(drop=True)
experiment_list.head()

In [ ]:
if not apply_test:
    experiment_list["metrics.test_mse_temp"] = np.NaN

scatter_values = [
    experiment_list["metrics.train_mse_temp"].values,
    experiment_list["metrics.validation_mse_temp"].values,
    experiment_list["metrics.test_mse_temp"].values,
    experiment_list["params.svm__C"].values.astype(np.float64),
    experiment_list["params.svm__epsilon"].values.astype(np.float64),
    experiment_list["params.svm__tol"].values.astype(np.float64),
    experiment_list["params.svm__gamma"].values.astype(np.float64),
]
df_experiment_svr = pd.DataFrame(
    np.transpose(scatter_values),
    columns=[
        "Train MSE in K",
        "Validation MSE in K",
        "Test MSE in K",
        "C",
        "Epsilon",
        "Tolerance",
        "Gamma",
    ],
)

In [ ]:
## limit the error if necessary
# df_experiment_svr[df_experiment_svr["Test MSE in K"] > 10] = np.nan

In [ ]:
# df_experiment_svr = df_experiment_svr.dropna()

In [ ]:
corr = df_experiment_svr.corr()
corr.style.background_gradient(cmap="turbo")

In [ ]:
# fig, axs = plt.subplots(7, 7, figsize=(15*cm, 15*cm),sharex=True)
# axs = pd.plotting.scatter_matrix(df_experiment_svr, ax=axs, diagonal='kde')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12 * cm, 12 * cm), sharex=True)

variables = ["C", "Epsilon", "Tolerance", "Gamma"]

for variable_idx, variable in enumerate(variables):
    plot_column = np.floor(variable_idx / 2).astype("int")
    plot_row = variable_idx - 2 * plot_column

    concave_hull_ratio = 0.25
    if apply_test:
        min_error = np.nanmin(
            [
                df_experiment_svr["Train MSE in K"].values,
                df_experiment_svr["Validation MSE in K"].values,
                df_experiment_svr["Test MSE in K"].values,
            ]
        )
    else:
        min_error = np.nanmin(
            [
                df_experiment_svr["Train MSE in K"].values,
                df_experiment_svr["Validation MSE in K"].values,
            ]
        )

    points = np.vstack(
        (df_experiment_svr["Train MSE in K"].values, df_experiment_svr[variable].values)
    ).T
    points = points[~np.isnan(points).any(axis=1)]
    # axs[plot_column,plot_row].scatter(df_experiment_svr["Train MSE in K"].values,df_experiment_svr[variable].values,c=eisplot.rwth_colors.colors[('blue', 100)], alpha=0.1,marker='.')
    points_hull = np.exp(
        np.array(
            shapely.concave_hull(
                shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
            ).exterior.coords
        )
    )
    axs[plot_column, plot_row].fill(
        points_hull[:, 0],
        points_hull[:, 1],
        color=eisplot.rwth_colors.colors[("petrol", 100)],
        alpha=0.5,
    )

    points = np.vstack(
        (
            df_experiment_svr["Validation MSE in K"].values,
            df_experiment_svr[variable].values,
        )
    ).T
    points = points[~np.isnan(points).any(axis=1)]
    # axs[plot_column,plot_row].scatter(df_experiment_svr["Validation MSE in K"].values,df_experiment_svr[variable].values,c=eisplot.rwth_colors.colors[('orange', 100)], alpha=0.1,marker='.')
    points_hull = np.exp(
        np.array(
            shapely.concave_hull(
                shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
            ).exterior.coords
        )
    )
    axs[plot_column, plot_row].fill(
        points_hull[:, 0],
        points_hull[:, 1],
        color=eisplot.rwth_colors.colors[("turqoise", 100)],
        alpha=0.5,
    )
    if apply_test:
        points = np.vstack(
            (
                df_experiment_svr["Test MSE in K"].values,
                df_experiment_svr[variable].values,
            )
        ).T
        points = points[~np.isnan(points).any(axis=1)]
        # axs[plot_column,plot_row].scatter(df_experiment_svr["Test MSE in K"].values,df_experiment_svr[variable].values,c=eisplot.rwth_colors.colors[('lavender', 100)], alpha=0.1,marker='.')
        points_hull = np.exp(
            np.array(
                shapely.concave_hull(
                    shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
                ).exterior.coords
            )
        )
        axs[plot_column, plot_row].fill(
            points_hull[:, 0],
            points_hull[:, 1],
            color=eisplot.rwth_colors.colors[("blue", 100)],
            alpha=0.5,
        )

    axs[plot_column, plot_row].set_ylabel(variable)
    axs[plot_column, plot_row].set_yscale("log")
    axs[plot_column, plot_row].set_xscale("log")
    axs[plot_column, plot_row].grid()
    axs[plot_column, plot_row].set_xlim([min_error * 0.8, 200])


legend_elements = [
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("petrol", 100)],
        label="train",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("turqoise", 100)],
        label="validation",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("blue", 100)],
        label="test",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle=":",
        color=eisplot.rwth_colors.colors[("darkred", 100)],
        label="selected value",
        alpha=1.0,
    ),
]
if not apply_test:
    legend_elements.pop(2)
fig.legend(
    handles=legend_elements,
    loc="upper center",
    scatterpoints=1,
    prop={"size": 8},
    ncol=4,
)


axs[1, 0].set_xlabel("MSE in K", size=8)
axs[1, 1].set_xlabel("MSE in K", size=8)
fig.tight_layout()

x_values = np.array(axs[0, 0].get_xlim()) * 0.95

axs[0, 0].plot(
    x_values,
    [float(run_eval.data.params["svm__C"]), float(run_eval.data.params["svm__C"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[0, 1].plot(
    x_values,
    [
        float(run_eval.data.params["svm__epsilon"]),
        float(run_eval.data.params["svm__epsilon"]),
    ],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[1, 0].plot(
    x_values,
    [float(run_eval.data.params["svm__tol"]), float(run_eval.data.params["svm__tol"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[1, 1].plot(
    x_values,
    [
        float(run_eval.data.params["svm__gamma"]),
        float(run_eval.data.params["svm__gamma"]),
    ],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)

fig.subplots_adjust(top=0.9)

In [ ]:
plot_name = (
    name_of_the_feature
    + ""
    + "_ExpID_"
    + mlflow.get_experiment_by_name(experiment_name).experiment_id
    + "_RunID_"
    + logged_model
)
plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

In [ ]:
destination_filepath = (
    r"./mlruns/"
    + name_of_this_run
    + "_ExpID_"
    + mlflow.get_experiment_by_name(experiment_name).experiment_id
    + ".parquet"
)
experiment_list.to_parquet(destination_filepath, compression="gzip", index=True)